In [1]:
import os
os.chdir('..')

import json
from workflows.skeletons import *
from utils import visualize, os_lib

os.environ['LANG'] = 'ch'
from templates.make_sentences import *

In [2]:
# 设置原始输入
obj = dict(
    user_id='test',
    query='今天的天气怎么样',
)

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults


class MakeTools(Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        # https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/
        # https://app.tavily.com/home
        # export TAVILY_API_KEY='xxx'
        tavily = TavilySearchResults(max_results=5)

        # 这里定义了3个工具，但这里3个工具其实都调用了同一个方法
        self.access_tools = {
            'tavily1': dict(
                func=tavily,
                desc='这是一个专门用来搜索音乐的搜索引擎',
                args=[dict(
                    name='input',
                    description='搜索关键词',
                    type='string'
                )]
            ),
            'tavily2': dict(
                func=tavily,
                desc='这是一个专门用来搜索烹饪美食的搜索引擎',
                args=[dict(
                    name='input',
                    description='搜索关键词',
                    type='string'
                )]
            ),
            'tavily3': dict(
                func=tavily,
                desc='这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索',
                args=[dict(
                    name='input',
                    description='搜索关键词',
                    type='string'
                )]
            ),
        }

    def on_process(self, obj, **kwargs):
        obj.update(access_tools=self.access_tools)
        return obj

# 增加工具
make_tools = MakeTools()
obj = make_tools(obj)

In [4]:
def log(title, content):
    if isinstance(content, dict):
        content = json.dumps(content, indent=4, ensure_ascii=False)
    elif isinstance(content, list):
        _content = ''
        for c in content:
            _content += str(c) + '\n'
        content = _content

    print(
        f"{visualize.TextVisualize.highlight_str(title)}\n"
        f"==========\n"
        f"{content}\n"
        f"----------\n"
    )

In [5]:
class SearchHistory(Module):
    size=10

    def __init__(self, cacher: os_lib.BaseCacher, **kwargs):
        super().__init__(**kwargs)
        self.cacher = cacher

    def on_process(self, obj, **kwargs):
        user_id = obj['user_id']
        # 历史信息可能会超长，请自行增加如截断、提取等策略，减少历史信息
        history = self.cacher.get_batch(key=user_id, size=self.size, search_type=cacher.LATEST)
        obj.update(history=history)
        return obj

# 简单演示，将历史数据缓存进内存里面
cacher = os_lib.MemoryCacher(verbose=False)
search_history = SearchHistory(cacher)
obj = search_history(obj)


In [6]:
class MakeMessages(Module):
    """以messages的形式输入"""
    def on_process(self, obj, debug=False, **kwargs):
        query = obj['query']
        access_tools = obj['access_tools']

        system = make_skeleton(
            additional_prompts=[
                make_today_date(),
                make_location('广州'),
                make_encourage(),
                make_access_tools(access_tools),
                make_thought_chain(list(access_tools.keys())),
            ]
        )

        messages = [
            {'role': 'system', 'content': system},
            {'role': 'user', 'content': query},
        ]

        if debug:
            log('模型输入', messages)

        obj.update(post_kwargs=dict(
            messages=messages
        ))
        return obj

make_messages = MakeMessages()
tmp = make_messages(obj, debug=True)

模型输入
{'role': 'system', 'content': '\n今天是2024-07-03。现在在广州。请尽你所能去回答问题。\n这是一些可用的工具，请从中挑选适用的工具解决问题:\ntavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]\ntavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]\ntavily3: 这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]\n\n请一步一步地使用以下的格式进行思考:\n\nQuestion: 你必须回答的问题\nThought: 你必须思考你应该怎么做\nAction: 你下一步采取的行动，你可以使用 [tavily1, tavily2, tavily3] 提供的工具\nAction Input: 你下一步行动中使用的工具的输入\nObservation: 你下一步行动的结果返回\n... (Thought/Action/Action Input/Observation 这些操作步骤可以重复多次)\nThought: 我现在知道怎么回答这个问题了\nFinal Answer: 上述的问题的最终的答案\n\n开始!\n\n'}
{'role': 'user', 'content': '今天的天气怎么样'}

----------



In [7]:
class MakePrompt(Module):
    """以prompt的形式输入"""
    def on_process(self, obj, first_query=True, debug=False, **kwargs):
        query = obj['query']
        history = obj.get('history', [])
        access_tools = obj['access_tools']

        if first_query:
            query = make_query(query)
        else:
            thought, action, action_input, observation = obj['thought'], obj['action'], obj['action_input'], obj['observation']
            query = make_thought_chain_query(query, thought, action, action_input, observation)

        prompt = make_skeleton(
            additional_prompts=[
                make_today_date(),
                make_location('广州'),
                make_encourage(),
                make_access_tools(access_tools),
                make_chat_history(history) if history else '',
                make_thought_chain(list(access_tools.keys())),
            ],
            query=query,
        )

        if debug:
            log('模型输入', prompt)

        obj.update(post_kwargs=dict(prompt=prompt))
        return obj

make_prompt = MakePrompt()
obj = make_prompt(obj, debug=True)

模型输入

今天是2024-07-03。现在在广州。请尽你所能去回答问题。
这是一些可用的工具，请从中挑选适用的工具解决问题:
tavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily3: 这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]

请一步一步地使用以下的格式进行思考:

Question: 你必须回答的问题
Thought: 你必须思考你应该怎么做
Action: 你下一步采取的行动，你可以使用 [tavily1, tavily2, tavily3] 提供的工具
Action Input: 你下一步行动中使用的工具的输入
Observation: 你下一步行动的结果返回
... (Thought/Action/Action Input/Observation 这些操作步骤可以重复多次)
Thought: 我现在知道怎么回答这个问题了
Final Answer: 上述的问题的最终的答案

开始!

Question: 今天的天气怎么样


----------



In [8]:
from models.bailian import Model

class LLM(Model):
    def on_process_end(self, obj, debug=False, **kwargs):
        if debug:
            log('模型输出', obj['content'])
        return obj

# export API_KEY='xxx'
llm = LLM(
    api_key=os.environ.get('API_KEY'),
    # llm_type='qwen2-7b-instruct'
)

obj = llm(obj, debug=True)


模型输出
Thought: 由于提供的工具中没有直接查询天气的搜索引擎，我将使用 tavily3 这个通用搜索引擎来查找广州今天的天气。

Action: tavily3

Action Input: {"input": "广州 2024-07-03 天气"}


----------



In [9]:
import re

class ParseThoughtChain(Module):
    def on_process(self, obj, debug=False, **kwargs):
        content = obj['content']
        access_tools = obj['access_tools']
        
        """这里请自行增加异常判断"""
        thought = re.search('Thought: (.+)', content).group(1)
        action = re.search('Action: (.+)', content).group(1)
        action_input = re.search('Action Input: (.+)', content).group(1)

        for k in access_tools:
            if k in action:
                action = k
                break
        else:
            raise
        
        try:
            action_input = json.loads(action_input)
        except:
            action_input = {'input': action_input}

        if debug:
            log('返回处理', f'{thought = }\n{action = }\n{action_input = }')

        obj.update(
            thought=thought,
            action=action,
            action_input=action_input
        )

        return obj
    
    
parse_thought_chain = ParseThoughtChain()
obj = parse_thought_chain(obj, debug=True)

返回处理
thought = '由于提供的工具中没有直接查询天气的搜索引擎，我将使用 tavily3 这个通用搜索引擎来查找广州今天的天气。'
action = 'tavily3'
action_input = {'input': '广州 2024-07-03 天气'}
----------



In [10]:
class ActionTools(Module):
    def on_process(self, obj, debug=False, **kwargs):
        access_tools = obj['access_tools']
        action_input = obj['action_input']
        action = obj['action']

        observation = access_tools[action]['func'].invoke(**action_input)
        if debug:
            log('搜索返回', observation)

        obj.update(observation=observation)
        return obj
    
action_tools = ActionTools()
obj = action_tools(obj, debug=True)

搜索返回
{'url': 'https://lishi.tianqi.com/guangzhou/202407.html', 'content': '全球天气网（www.tianqi.com）提供广州7月份天气数据，广州2024年7月份历史天气记录，广州7月份气温走势、风向等数据。 了解广州过去7月份天气，让您7月出行更方便。 ... (07/03) 日期. 最高气温 ... 风向. 2024-07-01 星期一 . 31℃ 27℃ 中雨. 西南风 3级. 查看更多. 广州 ...'}
{'url': 'https://www.tianqishi.com/guangzhou/20240703.html', 'content': '广州未来一周天气预报. 2024-07-04星期四：雷阵雨转多云,气温26~34℃,东风1级,气压1007hPa,相对湿度80%. 2024-07-05星期五：多云,气温26~34℃,东风1级,气压1006hPa,相对湿度75%. ... 2024-07-03 01:00: 28℃ ...'}
{'url': 'http://www.nmc.cn/publish/forecast/AGD/guangzhou.html', 'content': '广州天气预报 ; 省份： 城市： ... 07/03 周三 . 阵雨 . 无持续风向 . 微风 . 34℃ 28℃ 多云 . 无持续风向 . 微风 . 07/04 周四 . 多云 . 无持续风向 . 微风 . 35℃ ... 全球灾害性天气监测月报 . fy-4b . 雷达拼图 . 推荐服务 . 世界气象中心(北京) 台风网 . 气象期刊 .'}
{'url': 'https://www.meteoprog.com/cn/weather/Guangzhou/month/july/', 'content': '广州 七月 的天气。⛅ 广州 (中国) 七月, 2024 准确的天气预报。☀️ 1-15天内早晨，白天，傍晚，夜晚的详细天气预报。气象图，气象监控系统，天气预报对健康的影响。天气微件。广州的实际天气'}
{'url': 'http://www.tqyb.com.cn/', 'content': '广州市气象台2024年06月28日 16时30分发布. 短时预报 广州市气象台2024年06月28日

In [11]:
# 再请求一次模型
obj = make_prompt(obj, first_query=False, debug=True)
obj = llm(obj, debug=True)

模型输入

今天是2024-07-03。现在在广州。请尽你所能去回答问题。
这是一些可用的工具，请从中挑选适用的工具解决问题:
tavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily3: 这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]

请一步一步地使用以下的格式进行思考:

Question: 你必须回答的问题
Thought: 你必须思考你应该怎么做
Action: 你下一步采取的行动，你可以使用 [tavily1, tavily2, tavily3] 提供的工具
Action Input: 你下一步行动中使用的工具的输入
Observation: 你下一步行动的结果返回
... (Thought/Action/Action Input/Observation 这些操作步骤可以重复多次)
Thought: 我现在知道怎么回答这个问题了
Final Answer: 上述的问题的最终的答案

开始!

Question: 今天的天气怎么样
Thought: 由于提供的工具中没有直接查询天气的搜索引擎，我将使用 tavily3 这个通用搜索引擎来查找广州今天的天气。
Action: tavily3
Action Input: {'input': '广州 2024-07-03 天气'}
Observation: [{'url': 'https://lishi.tianqi.com/guangzhou/202407.html', 'content': '全球天气网（www.tianqi.com）提供广州7月份天气数据，广州2024年7月份历史天气记录，广州7月份气温走势、风向等数据。 了解广州过去7月份天气，让您7月出行更方便。 ... (07/03) 日期. 最高气

In [12]:
class ParseResult(Module):
    """解析模型返回"""
    def on_process(self, obj, debug=False, **kwargs):
        content = obj['content']
        answer = re.search('Final Answer: (.+)', content, re.DOTALL).group(1)
        
        if debug:
            log('查询返回', answer)
        obj.update(answer=answer)
        return obj

parse_result = ParseResult()
obj = parse_result(obj, debug=True)

查询返回
2024年7月3日广州的天气预计为阵雨，气温在28℃到34℃之间，风向不定，风力为微风级别。请记得带伞出门。
----------



In [13]:
import time

# 缓存进历史
def cache_history(obj, **kwargs):
    answer = obj['answer']
    query = obj['query']
    user_id = obj['user_id']
    history = [query, answer]
    time_str = time.time()
    cacher.cache_one(history, (user_id, time_str))

cache_history(obj)
cacher.get_batch(size=10)

[['今天的天气怎么样', '2024年7月3日广州的天气预计为阵雨，气温在28℃到34℃之间，风向不定，风力为微风级别。请记得带伞出门。']]

In [14]:
class Loop(LoopPipeline):
    def check(self, obj, it=None, **kwargs):
        content = obj['content']
        return 'Final Answer' in content
    
    def gen_kwargs(self, obj, **kwargs):
        kwargs.update(first_query=False)
        return kwargs
    
# 串成一条完整的流水线
pipeline = Pipeline(
    make_tools,
    search_history,
    make_prompt,
    llm,
    Loop(
        parse_thought_chain,
        action_tools,
        make_prompt,
        llm,
    ),
    parse_result
)
pipeline.register_success_callback(cache_history)

In [15]:
# 第二轮查询
obj = dict(
    user_id='test',
    query='需要带伞吗',
)
obj = pipeline(obj, debug=True)

模型输入

今天是2024-07-03。现在在广州。请尽你所能去回答问题。
这是一些可用的工具，请从中挑选适用的工具解决问题:
tavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily3: 这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]

这是以往的历史信息:

Question: 今天的天气怎么样
Answer: 2024年7月3日广州的天气预计为阵雨，气温在28℃到34℃之间，风向不定，风力为微风级别。请记得带伞出门。

请一步一步地使用以下的格式进行思考:

Question: 你必须回答的问题
Thought: 你必须思考你应该怎么做
Action: 你下一步采取的行动，你可以使用 [tavily1, tavily2, tavily3] 提供的工具
Action Input: 你下一步行动中使用的工具的输入
Observation: 你下一步行动的结果返回
... (Thought/Action/Action Input/Observation 这些操作步骤可以重复多次)
Thought: 我现在知道怎么回答这个问题了
Final Answer: 上述的问题的最终的答案

开始!

Question: 需要带伞吗


----------

模型输出
Thought: 根据之前提供的天气信息，今天广州的天气预计为阵雨，所以可以判断是否需要带伞。
Final Answer: 是的，根据今天广州的天气预报显示有阵雨，建议出门携带雨伞以防淋湿。
----------

查询返回
是的，根据今天广州的天气预报显示有阵雨，建议出门携带雨伞以防淋湿。
----------



In [16]:
# 第三轮查询
obj = dict(
    user_id='test',
    query='列10首适合今天天气听的周杰伦的歌曲',
)
obj = pipeline(obj, debug=True)

模型输入

今天是2024-07-03。现在在广州。请尽你所能去回答问题。
这是一些可用的工具，请从中挑选适用的工具解决问题:
tavily1: 这是一个专门用来搜索音乐的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily2: 这是一个专门用来搜索烹饪美食的搜索引擎 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]
tavily3: 这是一个搜索引擎，如果实在找不到合适的搜素工具可以使用这个工具进行搜索 s.args: [{"name": "input", "description": "搜索关键词", "type": "string"}]

这是以往的历史信息:

Question: 今天的天气怎么样
Answer: 2024年7月3日广州的天气预计为阵雨，气温在28℃到34℃之间，风向不定，风力为微风级别。请记得带伞出门。

Question: 需要带伞吗
Answer: 是的，根据今天广州的天气预报显示有阵雨，建议出门携带雨伞以防淋湿。

请一步一步地使用以下的格式进行思考:

Question: 你必须回答的问题
Thought: 你必须思考你应该怎么做
Action: 你下一步采取的行动，你可以使用 [tavily1, tavily2, tavily3] 提供的工具
Action Input: 你下一步行动中使用的工具的输入
Observation: 你下一步行动的结果返回
... (Thought/Action/Action Input/Observation 这些操作步骤可以重复多次)
Thought: 我现在知道怎么回答这个问题了
Final Answer: 上述的问题的最终的答案

开始!

Question: 列10首适合今天天气听的周杰伦的歌曲


----------

模型输出
Thought: 我需要找到适合雨天听的周杰伦的歌曲，最适合的工具应该是音乐搜索引擎tavily1。
Action: tavily1
Action Input: {"input": "周杰伦 雨天歌曲"}

----------

返回处理
thought = '我需